In [18]:
from nltk.tokenize import TweetTokenizer  # módulo para tokenizar los tweets
from nltk.corpus import stopwords  # lista de palabras basura
import unicodedata  # módulo para tratar caracteres unicode
import language_check  # módulo para corregir los tweets
from preprocessor.api import clean  # módulo para limpiar tweets INSTALAR: pip install tweet-preprocessor
import re # Para trabajar con texto
tknzr = TweetTokenizer()  # selecciona el tokenizador para tweets
tool = language_check.LanguageTool('es')  # trabajador de language_check para correjir

In [63]:
# Hacer esto, importante para bajar los data sets de NLTK!

import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [19]:
def strip_accents(s):
    """
    Quita los acentos para que el tokenizador de tweets no parta las palabras con tildes en 2.
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def remove_stopwords(sentence, language):
    """
    Dada una frase, devuelve la frase tokenizada y sin palabras basura
    """
    return [ token for token in tknzr.tokenize(strip_accents(sentence)) if token.lower() not in stopwords.words(language)]

In [20]:
frase = "estamos trists trabajando en análisis de sentimientos"
tokens = remove_stopwords(frase, "spanish")
print(tokens)

['trists', 'trabajando', 'analisis', 'sentimientos']


In [21]:
def correct_word(word):
    """
    Corrije las palabras si es necesario
    """
    matches = tool.check(word)
    return language_check.correct(word, matches)

In [22]:
for palabra in tokens:
    print(correct_word(palabra))

trisas
Trabajando
análisis
Sentimientos


In [23]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("spanish")

In [37]:
def clean_correct_tokenize(text):
    """
    Hace todo el procedimiento de limpiar, correjir, tokenizar, y sacar la raíz de las palabras.
    """
    text = re.sub(r'[^\w\s]','',text)
    clean_ = clean(text)
    matches = tool.check(clean_)
    corrected = language_check.correct(text, matches).lower()
    no_stop_words = remove_stopwords(corrected, 'spanish')
    tokens = [stemmer.stem(correct_word(word).lower()) for word in no_stop_words]
    return tokens

In [38]:
ejemplo = clean_correct_tokenize(frase)
print(ejemplo)

['tris', 'trabaj', 'analisis', 'sentimient']


In [65]:
# Abrir un archivo con texto

with open("comentarios_p.txt", "r", encoding='latin-1') as f:
    texto = f.readlines()
    
frases_preprocesadas = [] 
frases_desnudas = []

for sentence in texto: 
    tokenizado = clean_correct_tokenize(sentence)
    frases_preprocesadas.append(tokenizado)
    frases_desnudas.append(sentence)

In [46]:
with open("comentarios_preprocesados.txt", "w") as f:
    for frase in frases_preprocesadas:
        nueva_frase = " ".join(frase)
        f.write(nueva_frase+'\n')

In [47]:
from nltk import FreqDist, tokenize
with open("comentarios_preprocesados.txt", "r") as f:
    texto = f.read()
    palabras_preprocesadas = tokenize.word_tokenize(texto)
    fdist = FreqDist(palabras_preprocesadas)

In [51]:
# Palabras más comunes:

fdist.most_common(50)

[('univers', 108),
 ('estudi', 85),
 ('instal', 84),
 ('opin', 83),
 ('relacion', 72),
 ('profesor', 68),
 ('buen', 66),
 ('bosqu', 54),
 ('algun', 32),
 ('entabl', 31),
 ('espaci', 26),
 ('realiz', 26),
 ('mejor', 25),
 ('quej', 22),
 ('falt', 21),
 ('segur', 21),
 ('academ', 21),
 ('parec', 20),
 ('consej', 18),
 ('mas', 18),
 ('brind', 16),
 ('maton', 15),
 ('calid', 15),
 ('consider', 15),
 ('vez', 14),
 ('mayor', 14),
 ('recib', 14),
 ('evalu', 14),
 ('institu', 14),
 ('exist', 14),
 ('respuest', 13),
 ('si', 13),
 ('satisfech', 13),
 ('hac', 12),
 ('alumn', 12),
 ('dieron', 12),
 ('inquietud', 12),
 ('pued', 11),
 ('activ', 10),
 ('tem', 10),
 ('gust', 10),
 ('comput', 10),
 ('trabaj', 9),
 ('general', 9),
 ('estan', 9),
 ('agrad', 9),
 ('clas', 9),
 ('dud', 8),
 ('tal', 8),
 ('aunqu', 8)]

In [52]:
# Semillas:

positivas = [ 'buen', 'satisfech', 'mejor', 'gust', 'agrad']
negativas = ['quej', 'falt', 'maton' , 'dud']

In [42]:
from gensim.models import Word2Vec

model = Word2Vec(frases_preprocesadas, size=15, window=2, min_count=3, workers=4)

In [43]:
len(model.wv.vocab)

149

In [60]:
model.wv['no']

KeyError: "word 'no' not in vocabulary"

In [54]:
new_positivas = []
new_negativas = []
for pos in positivas:
    similar = model.wv.most_similar(positive=[pos], topn=3)
    for palabra in similar:
        palabra = palabra[0]
        if len(palabra)>1:
            new_positivas.append(palabra)
            
for neg in negativas:
    similar = model.wv.most_similar(positive=[neg], topn=3)
    for palabra in similar:
        palabra = palabra[0]
        if len(palabra)>1:
            new_negativas.append(palabra)

In [55]:
positivas += new_positivas
negativas += new_negativas

In [57]:
positivas = list(set(positivas))
negativas = list(set(negativas))

In [59]:
negativas

['escuch',
 'much',
 'mes',
 'maton',
 'instal',
 'sin',
 'ejempl',
 'obvi',
 'poc',
 'quej',
 'bosqu',
 'van',
 'dud',
 'falt',
 'institu']

In [62]:
polaridades = []

for frase in frases_preprocesadas:
    polaridad = 0
    for palabra in frase:
        try:
            model.wv[palabra]
            if palabra in positivas:
                polaridad += 1
            elif palabra in negativas:
                polaridad -= 1
        except:
            pass
    polaridades.append(polaridad)

In [69]:
frases_desnudas[0]

'¿Cómo evaluaría la calidad de la consejería académica que recibió en la institución? Me gustan, me siento muy cómodo pero algunos baños del ¿Cómo evaluaría la calidad de la consejería académica que recibió en la institución? séptimo piso del E son insoportables por el olor a orines.\n'

In [68]:
polaridades[1]

-1